In [1]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/
# 显示cell运行时长
%load_ext klab-autotime

features  lost+found  models  res


In [1]:
!ls -lh /home/kesci/input/bytedance/first-round/

total 9.0G
-rw-r--r-- 1 kesci 1000 426M May 22 04:05 test.csv
-rw-r--r-- 1 kesci 1000 8.6G May 16 20:15 train.csv


In [3]:
!ls -lh /home/kesci/work/features/

total 30G
-rw-r--r-- 1 kesci users 6.9M Jun 18 13:41 appear_times.joblib
-rw-r--r-- 1 kesci users  11M Jun 22 20:41 bys_ctr.joblib
-rw-r--r-- 1 kesci users 7.8M Jun 22 20:41 bys_ctr_q.joblib
-rw-r--r-- 1 kesci users  34M Jun 22 18:52 doc2vec_50_10.model
-rw-r--r-- 1 kesci users 2.2G Jun 22 18:52 doc2vec_50_10.model.docvecs.vectors_docs.npy
-rw-r--r-- 1 kesci users  97M Jun 22 18:52 doc2vec_50_10.model.trainables.syn1neg.npy
-rw-r--r-- 1 kesci users  45M Jun 22 18:52 doc2vec_50_10.model.trainables.vectors_docs_lockf.npy
-rw-r--r-- 1 kesci users  97M Jun 22 18:52 doc2vec_50_10.model.wv.vectors.npy
-rw-r--r-- 1 kesci users  46M Jun 18 05:20 doc2vec_50.model
-rw-r--r-- 1 kesci users 3.3G Jun 18 05:19 doc2vec_50.model.docvecs.vectors_docs.npy
-rw-r--r-- 1 kesci users 132M Jun 18 05:19 doc2vec_50.model.trainables.syn1neg.npy
-rw-r--r-- 1 kesci users  67M Jun 18 05:19 doc2vec_50.model.trainables.vectors_docs_lockf.npy
-rw-r--r-- 1 kesci users 132M Jun 18 05:19 doc2vec_50.model.wv.vectors.npy


In [3]:
!pwd

/home/kesci/work


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import lightgbm as lgb
import csv
import time
import json
import math
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from joblib import dump,load
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist
from sklearn.preprocessing import normalize
import gc

In [3]:
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=10000)
test_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',chunksize=10000)

In [3]:
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list
print(len(test_list))

5000000


### 加载

In [2]:
tfIdf = load('features/TfIdf.joblib')
d2vmodel = Doc2Vec.load("features/doc2vec_50_10.model")
bys_ctr = load('features/bys_ctr.joblib')
num_title = load('features/num_title.joblib')

bys_ctr = load('features/bys_ctr.joblib')
bys_ctr_q = load('features/bys_ctr_q.joblib')

num_tlast = load('features/num_tlast.joblib')
num_qlast = load('features/num_qlast.joblib')

In [3]:
print(len(bys_ctr),len(bys_ctr_q),len(num_tlast),len(num_qlast))

211103 151668 169844 145760


## tfidf

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=1000000)
test_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',chunksize=1000000)
texts = []
tmpid = -1
for i,chunk in enumerate(train_chunks):
    print(i)
    if i>=95:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for tmp in train:
            if tmp[0] != tmpid:
                texts.append(tmp[1])
                tmpid = tmp[0]
            texts.append(tmp[3])
tmpid = -1
for i,chunk in enumerate(test_chunks):
    # if i>=100:
    #     break
    np_X_file = np.array(chunk)#np.ndarray()
    train=np_X_file.tolist()#list
    for tmp in train:
        if tmp[0] != tmpid:
            texts.append(tmp[1])
            tmpid = tmp[0]
        texts.append(tmp[3][:-1])
print('start')
tfIdf = TfidfVectorizer()
tfIdf.fit(texts)
dump(tfIdf, 'features/TfIdf.joblib')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
start


['features/TfIdf.joblib']

In [7]:
len(tfIdf.get_feature_names())

503893

In [5]:
tfIdf = load('features/TfIdf.joblib')

## 词向量

In [3]:
#文件
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=1000000)
test_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',chunksize=1000000)
texts = []
tmpid=-1
for i,chunk in enumerate(train_chunks):
    print(i)
    if i>=95:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for tmp in train:
            if tmp[0] != tmpid:
                texts.append(tmp[1].split(' '))
                tmpid = tmp[0]
            texts.append(tmp[3].split(' '))
    
tmpid=-1
for i,chunk in enumerate(test_chunks):
    np_X_file = np.array(chunk)#np.ndarray()
    train=np_X_file.tolist()#list
    for tmp in train:
        if tmp[0] != tmpid:
            texts.append(tmp[1].split(' '))
            tmpid = tmp[0]
        texts.append(tmp[3][:-1].split(' '))
print('done')
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
d2vmodel = Doc2Vec(documents, vector_size=50, window=2, min_count=1, epochs=10, workers=1)
d2vmodel.save("features/doc2vec_50_10.model")
print("done doc2vec")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
done
done doc2vec


In [9]:
len(texts)

17492864

In [14]:
d2vmodel.wv.most_similar(['1'])

[('5620', 0.8750630617141724),
 ('7176', 0.8454800844192505),
 ('1303', 0.8393603563308716),
 ('125', 0.8236160278320312),
 ('5795', 0.8217899799346924),
 ('519', 0.8118669986724854),
 ('4355', 0.8096941113471985),
 ('2817', 0.8059057593345642),
 ('650', 0.7930350303649902),
 ('152', 0.7865456342697144)]

## 统计特征计算

### 概率

In [4]:
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=5000000)
select_per = {}

select_times = {}
appear_times = {}

select_per_q = {}

select_times_q = {}
appear_times_q = {}
lasttitle = 0
tmp_title_num = 0
for i,chunk in enumerate(tqdm(train_chunks)):
    if i>=10:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for split_line in train:
            tmpx=[]
            query = split_line[1].split(' ')
            title = split_line[3].split(' ')
            sign = int(split_line[4])
            if sign == 0:
                for q in query:
                    if q in appear_times_q:
                        appear_times_q[q] += 1
                    else:
                        appear_times_q[q] = 1
                    if q in select_times_q:
                        pass
                    else:
                        select_times_q[q] = 0
                for q in title:
                    if q in appear_times:
                        appear_times[q] += 1
                    else:
                        appear_times[q] = 1
                    if q in select_times:
                        pass
                    else:
                        select_times[q] = 0
            if sign == 1:    
                for q in query:
                    if q in appear_times_q:
                        appear_times_q[q] += 1
                    else:
                        appear_times_q[q] = 1
                    if q in select_times_q:
                        select_times_q[q] += 1
                    else:
                        select_times_q[q] = 1
                for q in title:
                    if q in appear_times:
                        appear_times[q] += 1
                    else:
                        appear_times[q] = 1
                    if q in select_times:
                        select_times[q] += 1
                    else:
                        select_times[q] = 1 
for s in select_times:
    select_per[s] = select_times[s]/appear_times[s]
for s in select_times_q:
    select_per_q[s] = select_times_q[s]/appear_times_q[s]

20it [14:44, 79.18s/it]


In [14]:
dump(select_per,'features/select_per.joblib')
dump(select_times,'features/select_times.joblib')
dump(appear_times,'features/appear_times.joblib')

['features/appear_times.joblib']

In [13]:
select_per = load('features/select_per.joblib')
select_times = load('features/select_times.joblib')

In [15]:
len(select_per)

686716

### query对应title数

In [6]:
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=1000000)
num_title = {}
lasttitle = 0
tmp_title_num = 0
for i,chunk in enumerate(tqdm(train_chunks)):
    if i>=95:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for split_line in train:
            if lasttitle != split_line[1]:
                num_title[lasttitle] = tmp_title_num
                lasttitle = split_line[1]
                tmp_title_num=0
            tmp_title_num+=1    
num_title[split_line[1]] = tmp_title_num

lasttitle = 0
tmp_title_num = 0
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list
print(len(test_list))

for i,split_line in enumerate(tqdm(test_list)):
    if lasttitle != split_line[1]:
        num_title[lasttitle] = tmp_title_num
        lasttitle = split_line[1]
        tmp_title_num=0
    tmp_title_num+=1    
num_title[split_line[1]] = tmp_title_num

  2%|▏         | 77397/5000000 [00:00<00:06, 773955.98it/s]

5000000


100%|██████████| 5000000/5000000 [00:03<00:00, 1283255.37it/s]


In [8]:
dump(num_title,'features/num_title.joblib')

['features/num_title.joblib']

In [7]:
len(num_title)

1733044

## 结尾词特征

In [5]:
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=1000000)
num_tlast = {}
num_qlast = {}
for i,chunk in enumerate(tqdm(train_chunks)):
    if i>=95:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for split_line in train:
            tmpx=[]
            query = split_line[1].split(' ')
            title = split_line[3].split(' ')
            q_last = query[-1]
            t_last = title[-1]
            
            if q_last in num_qlast:
                num_qlast[q_last] += 1
            else:
                num_qlast[q_last] = 1
            if t_last in num_tlast:
                num_tlast[t_last] += 1
            else:
                num_tlast[t_last] = 1
                
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list
print(len(test_list))                
for i,split_line in enumerate(tqdm(test_list)):
    tmpx=[]
    query = split_line[1].split(' ')
    title = split_line[3][:-1].split(' ')
    q_last = query[-1]
    t_last = title[-1].strip()
    
    if q_last in num_qlast:
        num_qlast[q_last] += 1
    else:
        num_qlast[q_last] = 1
    if t_last in num_tlast:
        num_tlast[t_last] += 1
    else:
        num_tlast[t_last] = 1

100it [03:02,  4.12s/it]
  0%|          | 24209/5000000 [00:00<00:20, 242085.61it/s]

5000000


100%|██████████| 5000000/5000000 [00:16<00:00, 298293.30it/s]


In [6]:
dump(num_tlast,'features/num_tlast.joblib')
dump(num_qlast,'features/num_qlast.joblib')

['features/num_qlast.joblib']

In [17]:
len(num_tlast)

169844

### 贝叶斯

In [7]:
import time
import scipy.special as special
class BayesianSmoothing(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def sample(self, alpha, beta, num, imp_upperbound):
        sample = numpy.random.beta(alpha, beta, num)
#         print(sample)
        I = []
        C = []
        for clk_rt in sample:
            imp = imp_upperbound
            clk = imp * clk_rt
            I.append(imp)
            C.append(clk)
        return I, C

    def update(self, imps, clks, iter_num, epsilon):

        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(imps, clks, self.alpha, self.beta)
            if abs(new_alpha - self.alpha) < epsilon and abs(new_beta - self.beta) < epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta

    def __fixed_point_iteration(self, imps, clks, alpha, beta):
        numerator_alpha = 0.0
        numerator_beta = 0.0
        denominator = 0.0
#         print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        for i in range(0, len(imps), 1):    # 步长复用时去掉
            numerator_alpha += (special.digamma(clks[i] + alpha) - special.digamma(alpha))
            numerator_beta += (special.digamma(imps[i] - clks[i] + beta) - special.digamma(beta))
            denominator += (special.digamma(imps[i] + alpha + beta) - special.digamma(alpha + beta))
        return alpha * (numerator_alpha / denominator), beta * (numerator_beta / denominator)
def bys(I, C):
    bs = BayesianSmoothing(1, 1)
    # I, C = bs.sample(500, 500, 10, 1000)
    bs.update(I, C, 1000, 0.00001)   #
#     print(bs.alpha, bs.beta)
    ctr = []
    for i in range(len(I)):
        ctr.append((C[i] + bs.alpha) / (I[i] + bs.alpha + bs.beta))
    return ctr

In [29]:
bys_ctr = {}
for s in tqdm(select_per):
    if select_times[s] >= 10:
        bys_ctr[s] = 1
print(len(bys_ctr))

100%|██████████| 388715/388715 [00:00<00:00, 744050.90it/s]

75340


In [8]:
bys_ctr = {}
for s in tqdm(select_per):
    if select_times[s] >= 10:
        bys_ctr[s] = bys([select_times[s]],[select_per[s]])
len(bys_ctr)

100%|██████████| 686716/686716 [52:17<00:00, 218.88it/s]  


211103

In [9]:
for s in select_times_q:
    select_per_q[s] = select_times_q[s]/appear_times_q[s]
bys_ctr_q = {}
for s in tqdm(select_per_q):
    if select_times_q[s] >= 10:
        bys_ctr_q[s] = bys([select_times_q[s]],[select_per_q[s]])
len(bys_ctr_q)

100%|██████████| 443780/443780 [38:07<00:00, 194.00it/s]  


151668

In [10]:
dump(bys_ctr,'features/bys_ctr.joblib')
dump(bys_ctr_q,'features/bys_ctr_q.joblib')

['features/bys_ctr_q.joblib']

In [19]:
len(bys_ctr)

211103

In [27]:
bys_ctr = load('features/bys_ctr.joblib')
# bys_ctr_q = load('features/bys_ctr_q.joblib')

In [24]:
sorted(select_times.items(), key=lambda item: item[1],reverse=True)

[('668', 967),
 ('4029', 961),
 ('36', 900),
 ('3950', 732),
 ('4700', 617),
 ('2216', 537),
 ('1851', 475),
 ('2880', 444),
 ('1247', 434),
 ('1289', 416),
 ('2539', 414),
 ('270', 400),
 ('5789', 390),
 ('3167', 375),
 ('7683', 360),
 ('4997', 349),
 ('12583', 329),
 ('5413', 328),
 ('9329', 316),
 ('929', 310),
 ('340', 308),
 ('13296', 305),
 ('9381', 296),
 ('6389', 290),
 ('15874', 271),
 ('86', 270),
 ('70', 270),
 ('18', 269),
 ('19', 269),
 ('11', 268),
 ('71', 267),
 ('5762', 266),
 ('5488', 265),
 ('4144', 265),
 ('6131', 263),
 ('7620', 257),
 ('11094', 254),
 ('11931', 251),
 ('2049', 246),
 ('8300', 245),
 ('2924', 244),
 ('5960', 238),
 ('2377', 234),
 ('11627', 233),
 ('9296', 231),
 ('13758', 230),
 ('8056', 230),
 ('2313', 229),
 ('10875', 225),
 ('10343', 225),
 ('8732', 222),
 ('3902', 221),
 ('8614', 221),
 ('5610', 218),
 ('6015', 218),
 ('4696', 210),
 ('11007', 210),
 ('4385', 210),
 ('6505', 209),
 ('7187', 205),
 ('6402', 204),
 ('6686', 204),
 ('2094', 203),


## 特征函数

In [92]:
get_tfidf_Score(['12','23','36'])

{'12', '23', '36'}

In [4]:
#余弦距离和欧式距离
from scipy.stats import pearsonr,spearmanr,kendalltau
def w2v_d2v(vec1, vec2):
    npvec1, npvec2 = np.array(vec1), np.array(vec2)

    Cosine = npvec1.dot(npvec2)/(math.sqrt((npvec1**2).sum()) * math.sqrt((npvec2**2).sum()))
    Euclidean = math.sqrt(((npvec1-npvec2)**2).sum())
    return Cosine, Euclidean
    
#编辑距离
def edit_dis(word1,word2):
    n = len(word1)
    m = len(word2)
    # if one of the strings is empty
    if n * m == 0:
        return n + m
    # array to store the convertion history
    d = [ [0] * (m + 1) for _ in range(n + 1)]

    # init boundaries
    for i in range(n + 1):
        d[i][0] = i
    for j in range(m + 1):
        d[0][j] = j

    # DP compute 
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            left = d[i - 1][j] + 1
            down = d[i][j - 1] + 1
            left_down = d[i - 1][j - 1] 
            if word1[i - 1] != word2[j - 1]:
                left_down += 1
            d[i][j] = min(left, down, left_down)

    return d[n][m]
    
#pearson
def pearson(x,y):
    return pearsonr(x,y)[0]
def spearman(x,y):
    return spearmanr(x,y)[0]
def kendall(x,y):
    return kendalltau(x,y)[0]  
def minkowskiDisSim(x,y,p=3):
    '''
    明可夫斯基距离计算方法
    '''
    sumvalue=sum(pow(abs(a-b),p) for a,b in zip(x,y))
    tmp=1/float(p)
    return round(sumvalue**tmp,3)

def MahalanobisDisSim(x,y):
    '''
    马氏距离计算方法
    '''
    npvec1,npvec2=np.array(x),np.array(y)
    npvec=np.array([npvec1, npvec2])
    sub=npvec.T[0]-npvec.T[1]
    inv_sub=np.linalg.inv(np.cov(npvec1, npvec2))
    return math.sqrt(np.dot(inv_sub, sub).dot(sub.T))
def jaccardDisSim(x,y):
    '''
    杰卡德相似度计算
    '''
    res=len(set.intersection(*[set(x),set(y)]))
    union_cardinality=len(set.union(*[set(x),set(y)]))
    return res/float(union_cardinality)      
#词语交集数
def intersect(word1,word2):
    count = 0
    for w in word1:
        if w in word2:
            count+=1
    return count

#tfidf 相关特征
featureName = tfIdf.get_feature_names()
def get_tfidf_Score(words):
    tfIdfNameScore = {}
    tfIdfFeatures = tfIdf.transform([' '.join(words)])
    tfIdfScores = tfIdfFeatures.data
    # normalize
    for name, score in zip(tfIdfFeatures.indices, tfIdfScores):
        tfIdfNameScore[featureName[name]] = score
    NameScore = sorted(tfIdfNameScore.items(), key=lambda item: item[1], reverse=True)
    names5 = set()
    for name in NameScore[:5]:
        names5.add(name[0])
    names3 = set()
    for name in NameScore[:3]:
        names3.add(name[0])
    return names5,names3
#tfidf前n交集数    
def inter_tfidf(words1,words2):
    names1,names1_3 = get_tfidf_Score(words1)
    names2,names2_3 = get_tfidf_Score(words2)
    
    n = min(5,min(len(words1),len(words2)))
    if n != 0:
        ratio = 5/n
    else:
        ratio = 5
    count=0
    for i in names1:
        if i in names2:
            count+=1
            
    n = min(3,min(len(words1),len(words2)))
    if n != 0:
        ratio3 = 5/n
    else:
        ratio3 = 5
    count3=0
    for i in names1_3:
        if i in names2_3:
            count3+=1
    return count*ratio,count3*ratio3


def tongji(word1,word2,words1,words2):
    
    # -------------------------title--------------------------
    sumper = 0
    per = []
    num_hit = 0
    for w in words2:
        try:
            sumper += bys_ctr[w][0]
            per.append(bys_ctr[w][0])
            num_hit += 1
        except:
            pass
    try:
        maxper = max(per)
        minper = min(per)
        sumper_ratio = sumper/max(1,num_hit)
    except:
        maxper = np.nan
        minper = np.nan
        sumper_ratio = np.nan
        sumper = np.nan
    
    # -------------------------qiery--------------------------
    sumper_q=0
    per_q=[]
    num_hit_q = 0
    for w in words1:
        try:
            sumper_q += bys_ctr_q[w][0]
            per_q.append(bys_ctr_q[w][0])
            num_hit_q += 1
        except:
            pass
    try:
        maxper_q = max(per_q)
        minper_q = min(per_q)
        sumper_ratio_q = sumper_q/max(1,num_hit_q)
    except:
        maxper_q = np.nan
        minper_q = np.nan
        sumper_ratio_q = np.nan
        sumper_q = np.nan
    
    return num_title[word1], sumper_ratio, sumper, maxper, minper, sumper_ratio_q, sumper_q, maxper_q, minper_q

In [5]:
len(num_title)

1733044

In [6]:
bys_ctr['63'][0]

1.2206751405082567e-06

In [7]:
len(bys_ctr_q)

151668

## 特征计算

In [8]:
#训练集
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=5000000)
# d2vmodel = Doc2Vec.load("features/doc2vec.model")
x = []
y = []
c = 0
for i,chunk in enumerate(tqdm(train_chunks)):
    if i==19:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for split_line in train:
            c+=1
            if c%10000 == 0:
                print(c)
            tmpx=[]
            query = split_line[1].split(' ')
            title = split_line[3].split(' ')
            query_set = set(query)
            title_set = set(title)
            
            dv1 = d2vmodel.infer_vector(query)
            dv2 = d2vmodel.infer_vector(title)
            query_len = len(query)
            title_len = len(title)
            
            nt, per_ratio, sumper, maxper, minper, per_ratio_q, sumper_q, maxper_q, minper_q = tongji(split_line[1],split_line[3],query,title)
            q_last = query[-1]
            t_last = title[-1]
            tl = num_tlast[t_last]
            ql = num_qlast[q_last]
            # k_means = km[i]
            if split_line[1] in split_line[3]:
                iit = 1
            else:
                iit = 0
            
            cos, euc = w2v_d2v(dv1,dv2)
            edit = edit_dis(query,title)
            edit_ratio1 = edit/(query_len)
            edit_ratio2 = edit/(title_len)
            edit_ratio3 = edit/(query_len+title_len)
            pearson_corr = pearson(dv1,dv2)
            spearman_corr = spearman(dv1,dv2)
            kendall_corr = kendall(dv1,dv2)
            minkowskiDisSim_corr = minkowskiDisSim(dv1,dv2)
            MahalanobisDisSim_corr = MahalanobisDisSim(dv1,dv2)
            jaccardDisSim_corr = jaccardDisSim(dv1,dv2)
        
            inter_num = intersect(query_set,title_set)
            inter_tfidf_num5,inter_tfidf_num3 = inter_tfidf(query,title)
        
            # -------------------------------------------------------------------------------------
            tmpx.append(query_len) #长度
            tmpx.append(title_len) # -- 
            tmpx.append(title_len/query_len) # --
            tmpx.append(nt) #query的标题数量
            tmpx.append(per_ratio) #title的选择概率相加
            tmpx.append(sumper) #title概率加和
            tmpx.append(maxper) #title概率最大
            tmpx.append(minper) #title概率最小
            
            tmpx.append(per_ratio_q) #query的选择概率相加
            tmpx.append(sumper_q) #query概率加和
            tmpx.append(maxper_q) #query概率最大
            tmpx.append(minper_q) #query概率最小
            tmpx.append(ql)
            tmpx.append(tl)
            # tmpx.append(k_means)
            tmpx.append(iit)
            
            tmpx.append(cos) #余弦距离
            tmpx.append(euc)  #欧氏距离
            tmpx.append(edit) #编辑距离
            tmpx.append(edit_ratio1) #编辑距离比率
            tmpx.append(edit_ratio2)
            tmpx.append(edit_ratio3)
            tmpx.append(pearson_corr) #pearson相关系数
            tmpx.append(spearman_corr)
            tmpx.append(kendall_corr)
            tmpx.append(minkowskiDisSim_corr)
            tmpx.append(MahalanobisDisSim_corr)
            tmpx.append(jaccardDisSim_corr)
        
            tmpx.append(inter_num) #词交集数
            tmpx.append(inter_num/query_len) #词交集比率
            tmpx.append(inter_num/(query_len+title_len)) #词交集比率2
            tmpx.append(inter_tfidf_num5) #tfidf交集数
            tmpx.append(inter_tfidf_num3)
            
            x.append(tmpx)
            y.append(split_line[4])

19it [03:00,  9.52s/it]

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

20it [6:53:14, 7390.77s/it]


In [9]:
x_csv = pd.DataFrame(x)
x_csv.to_csv('features/x_623.csv',index=False)
dump(y,'features/y_623.joblib')
# x_csv = []

['features/y_623.joblib']

In [18]:
del(x_csv)
gc.collect()

200

In [7]:
#按每个query排序的特征
train_chunks = pd.read_csv('/home/kesci/input/bytedance/first-round/train.csv',chunksize=5000000)
tmpx = []
tmpid=-1
for j,chunk in enumerate(tqdm(train_chunks)):
    if j==19:
        np_X_file = np.array(chunk)#np.ndarray()
        train=np_X_file.tolist()#list
        for i,split_line in enumerate(train):
            if i == 0:
                tmpid = split_line[0]
            if split_line[0] != tmpid or i == 4999999:
                tmpid = split_line[0]
                l = len(tmpx)
                fl = len(tmpx[0])
                for j in range(fl): #每个特征都排序
                    sot = []
                    ttmpx = []
                    # chazhi = []
                    for k in range(l):        #获取query长度的单个特征
                        ttmpx.append(tmpx[k][j])
                    # zuida = 0
                    for q in range(l):
                        # if ttmpx[q]>zuida:
                        #     zuida = ttmpx[q]
                        c = 0
                        for m in range(l):  
                            if ttmpx[q] > ttmpx[m]:
                                c += 1
                        sot.append(c)
                    # for q in range(l):
                    #     chazhi.append(zuida-ttmpx[q])
                    for p in range(l):
                        x[i-l+p].append(sot[p])
                        # x[i-l+p].append(chazhi[p])
                tmpx=[]
            
            tmpx.append(x[i])
for m in range(32):
    x[4999998].append(0)
    # x[4999998].append(0)

20it [10:57, 154.21s/it]


In [9]:
del(train)
del(np_X_file)
gc.collect()

0

In [18]:
len(x)

4999999

In [16]:
len(x[0])

33

In [19]:
x = x[0:4999990]

In [20]:
y = y[0:4999990]

In [29]:
from collections import Counter
t = []
for tmp in x:
    t.append(len(tmp))
print(Counter(t))

Counter({64: 4999990})


In [7]:
for split_line in test_list:
    query = split_line[1].split(' ')
    title = split_line[3][:-1].split(' ')
    print(title)
    break

['11202', '184', '2346', '2527', '274', '383', '34', '1033', '156', '18', '50256', '19', '27', '11533', '7761', '1600', '2769\t']


In [10]:
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list
print(len(test_list))

5000000


In [11]:
#测试集
c = 0
x_test = []
for split_line in test_list:
    c+=1
    if c%50000 == 0:
        print(c)
    tmpx=[]
    query = split_line[1].split(' ')
    title = split_line[3][:-1].split(' ')
    query_set = set(query)
    title_set = set(title)
    
    dv1 = d2vmodel.infer_vector(query)
    dv2 = d2vmodel.infer_vector(title)
    query_len = len(query)
    title_len = len(title)
    nt, per_ratio, sumper, maxper, minper, per_ratio_q, sumper_q, maxper_q, minper_q = tongji(split_line[1],split_line[3],query,title)
    q_last = query[-1]
    t_last = title[-1].strip()
    tl = num_tlast[t_last]
    ql = num_qlast[q_last]
    # k_means = km[i]
    if split_line[1] in split_line[3]:
        iit = 1
    else:
        iit = 0
    
    cos, euc = w2v_d2v(dv1,dv2)
    edit = edit_dis(query,title)
    edit_ratio1 = edit/(query_len)
    edit_ratio2 = edit/(title_len)
    edit_ratio3 = edit/(query_len+title_len)
    pearson_corr = pearson(dv1,dv2)
    spearman_corr = spearman(dv1,dv2)
    kendall_corr = kendall(dv1,dv2)
    minkowskiDisSim_corr = minkowskiDisSim(dv1,dv2)
    MahalanobisDisSim_corr = MahalanobisDisSim(dv1,dv2)
    jaccardDisSim_corr = jaccardDisSim(dv1,dv2)

    inter_num = intersect(query_set,title_set)
    inter_tfidf_num5,inter_tfidf_num3 = inter_tfidf(query,title)

    # -------------------------------------------------------------------------------------
    tmpx.append(query_len) #长度
    tmpx.append(title_len) # -- 
    tmpx.append(title_len/query_len) # --
    tmpx.append(nt) #query的标题数量
    tmpx.append(per_ratio) #title的选择概率相加
    tmpx.append(sumper) #title概率加和
    tmpx.append(maxper) #title概率最大
    tmpx.append(minper) #title概率最小
    
    tmpx.append(per_ratio_q) #query的选择概率相加
    tmpx.append(sumper_q) #query概率加和
    tmpx.append(maxper_q) #query概率最大
    tmpx.append(minper_q) #query概率最小
    tmpx.append(ql)
    tmpx.append(tl)
    # tmpx.append(k_means)
    tmpx.append(iit)
    
    tmpx.append(cos) #余弦距离
    tmpx.append(euc)  #欧氏距离
    tmpx.append(edit) #编辑距离
    tmpx.append(edit_ratio1) #编辑距离比率
    tmpx.append(edit_ratio2)
    tmpx.append(edit_ratio3)
    tmpx.append(pearson_corr) #pearson相关系数
    tmpx.append(spearman_corr)
    tmpx.append(kendall_corr)
    tmpx.append(minkowskiDisSim_corr)
    tmpx.append(MahalanobisDisSim_corr)
    tmpx.append(jaccardDisSim_corr)

    tmpx.append(inter_num) #词交集数
    tmpx.append(inter_num/query_len) #词交集比率
    tmpx.append(inter_num/(query_len+title_len)) #词交集比率2
    tmpx.append(inter_tfidf_num5) #tfidf交集数
    tmpx.append(inter_tfidf_num3)
    
    x_test.append(tmpx)

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000


In [12]:
x_test_csv = pd.DataFrame(x_test)
x_test_csv.to_csv('features/x_test_623.csv',index=False)

In [13]:
x_test_csv

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,3,17,5.666667,4,0.000124,0.002100,0.001585,2.980030e-08,0.001131,0.003392,...,0.529604,0.364898,0.406,0.439526,0.0,3,1.000000,0.150000,3.333333,3.333333
1,3,15,5.000000,4,0.000167,0.002503,0.001585,2.980030e-08,0.001131,0.003392,...,0.396879,0.273469,0.346,1.506209,0.0,3,1.000000,0.166667,3.333333,1.666667
2,3,15,5.000000,4,0.000142,0.002125,0.001585,6.356990e-08,0.001131,0.003392,...,0.343385,0.245714,0.452,1.381275,0.0,3,1.000000,0.166667,5.000000,3.333333
3,3,20,6.666667,4,0.000090,0.001793,0.001585,2.980030e-08,0.001131,0.003392,...,0.007635,-0.000816,0.637,0.461643,0.0,3,1.000000,0.130435,3.333333,3.333333
4,4,15,3.750000,3,0.000022,0.000325,0.000089,2.980030e-08,0.000144,0.000578,...,0.296134,0.219592,0.269,2.505235,0.0,2,0.500000,0.105263,2.500000,1.666667
5,4,6,1.500000,3,0.000019,0.000116,0.000089,6.356990e-08,0.000144,0.000578,...,0.609316,0.446531,0.222,1.395038,0.0,2,0.500000,0.200000,2.500000,3.333333
6,4,13,3.250000,3,0.000054,0.000707,0.000476,6.356990e-08,0.000144,0.000578,...,0.516351,0.364898,0.348,1.235388,0.0,2,0.500000,0.117647,2.500000,1.666667
7,3,4,1.333333,8,0.000002,0.000007,0.000003,8.613904e-08,0.000014,0.000042,...,0.866699,0.684898,0.175,2.863823,0.0,2,0.666667,0.285714,3.333333,3.333333
8,3,8,2.666667,8,0.000003,0.000025,0.000006,2.980030e-08,0.000014,0.000042,...,0.951789,0.833469,0.347,0.669902,0.0,2,0.666667,0.181818,3.333333,1.666667
9,3,7,2.333333,8,0.000052,0.000364,0.000270,2.980030e-08,0.000014,0.000042,...,0.944970,0.825306,0.112,1.616837,0.0,2,0.666667,0.200000,3.333333,1.666667


In [19]:
del(x_test_csv)
gc.collect()

In [15]:
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list
print(len(test_list))
#按每个query排序的特征
tmpx = []
tmpid=-1
for i,split_line in enumerate(test_list):
    if i == 0:
        tmpid = split_line[0]
    if split_line[0] != tmpid or i == 4999999:
        tmpid = split_line[0]
        l = len(tmpx)
        fl = len(tmpx[0])
        for j in range(fl): #每个特征都排序
            sot = []
            # chazhi = []
            ttmpx = []
            for k in range(l):        #获取query长度的单个特征
                ttmpx.append(tmpx[k][j])
            # zuida = 0
            for q in range(l):
                c = 0
                for m in range(l):  
                    if ttmpx[q] > ttmpx[m]:
                        c += 1
                sot.append(c)
            # for q in range(l):
            #     chazhi.append(zuida-ttmpx[q])
            for p in range(l):
                x_test[i-l+p].append(sot[p])
                # x_test[i-l+p].append(chazhi[p])
        tmpx=[]
    
    tmpx.append(x_test[i])
for m in range(32):
    x_test[4999999].append(0)
    # x_test[4999999].append(0)

5000000


In [50]:
for m in range(32):
    x_test[4999999].append(0)

In [30]:
from collections import Counter
t = []
for tmp in x_test:
    t.append(len(tmp))
print(Counter(t))

Counter({64: 5000000})


In [14]:
x_test_csv = pd.read_csv('features/x_test_623.csv')
np_test_file = np.array(x_test_csv)#np.ndarray()
x_test=np_test_file.tolist()#list

In [4]:
len(x_test)

5000000

In [2]:
x_csv = pd.read_csv('features/x_623.csv')

In [6]:
x_csv

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,2,8,4.000000,3,0.000161,0.001286,0.000490,1.258846e-07,0.000574,0.001148,...,0.656567,0.467755,0.275,0.859353,0.0,2,1.000000,0.200000,5.000000,2.500000
1,2,7,3.500000,3,0.000314,0.002198,0.000839,9.498564e-06,0.000574,0.001148,...,0.300168,0.195102,0.391,0.718017,0.0,2,1.000000,0.222222,5.000000,2.500000
2,2,11,5.500000,3,0.000099,0.001087,0.000490,2.980030e-08,0.000574,0.001148,...,0.449700,0.322449,0.483,1.364371,0.0,1,0.500000,0.076923,2.500000,2.500000
3,7,8,1.142857,7,0.000577,0.004619,0.003965,1.268481e-07,0.000664,0.003987,...,0.880240,0.714286,0.160,1.201768,0.0,7,1.000000,0.466667,5.000000,5.000000
4,7,18,2.571429,7,0.000043,0.000730,0.000171,8.613904e-08,0.000664,0.003987,...,0.091861,0.056327,0.613,1.441650,0.0,5,0.714286,0.200000,2.000000,0.000000
5,7,8,1.142857,7,0.000128,0.000895,0.000377,1.003513e-05,0.000664,0.003987,...,0.900216,0.738776,0.179,1.887276,0.0,5,0.714286,0.333333,2.000000,0.000000
6,7,19,2.714286,7,0.000049,0.000873,0.000171,8.613904e-08,0.000664,0.003987,...,-0.012917,-0.010612,0.624,0.775476,0.0,5,0.714286,0.192308,3.000000,0.000000
7,7,17,2.428571,7,0.000065,0.001106,0.000296,4.079251e-07,0.000664,0.003987,...,0.602689,0.417143,0.384,0.201697,0.0,5,0.714286,0.208333,1.000000,0.000000
8,7,13,1.857143,7,0.000562,0.007309,0.003965,2.686677e-07,0.000664,0.003987,...,0.385930,0.265306,0.390,0.691540,0.0,7,1.000000,0.350000,3.000000,3.333333
9,7,12,1.714286,7,0.001352,0.016224,0.013186,1.268481e-07,0.000664,0.003987,...,0.777287,0.596735,0.299,0.218309,0.0,3,0.428571,0.157895,1.000000,0.000000


In [3]:
np_train_file = np.array(x_csv)#np.ndarray()
x=np_train_file.tolist()#list
print(len(x))
del(x_csv)
gc.collect()

In [4]:
y = load('features/y_623.joblib')

In [28]:
x = train_list
del(train_list)
gc.collect()
print(len(x))

4999999


In [6]:
len(x)

4999999

In [22]:
def model_lgb(X, Y, process_num, evalx, evaly, seed):
    trainx, evalx, trainy, evaly = train_test_split(X, Y, test_size=0.3, random_state=seed)
    X = pd.DataFrame(trainx)
    Y = pd.DataFrame(trainy)
    evalx = pd.DataFrame(evalx)
    evaly = pd.DataFrame(evaly)
    lgb_train = lgb.Dataset(X, Y)
    lgb_eval = lgb.Dataset(evalx, evaly, reference=lgb_train)
    
    params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['auc'], #xentropy
    'num_leaves': 31,
    # 'max_depth' : 8,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_seed': seed,
    'bagging_freq': 1,
    'seed':seed,
    # 'verbosity':10,
    'lambda_l1': 0.2,
    # 'lambda_l2': 2
    }
    print("Training lgb model....")
    gbm = lgb.train(params, lgb_train,num_boost_round=500, valid_sets=lgb_eval, early_stopping_rounds=100)
    print("Save model to "+process_num+".joblib")
    dump(gbm, "models/"+process_num+".joblib")
model_lgb(x, y, "lgb_v5", 0, 0, 1)

Training lgb model....
[1]	valid_0's auc: 0.66399
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.665057
[3]	valid_0's auc: 0.665545
[4]	valid_0's auc: 0.666123
[5]	valid_0's auc: 0.666398
[6]	valid_0's auc: 0.666646
[7]	valid_0's auc: 0.666981
[8]	valid_0's auc: 0.66715
[9]	valid_0's auc: 0.667398
[10]	valid_0's auc: 0.667719
[11]	valid_0's auc: 0.667885
[12]	valid_0's auc: 0.668078
[13]	valid_0's auc: 0.668275
[14]	valid_0's auc: 0.66838
[15]	valid_0's auc: 0.668553
[16]	valid_0's auc: 0.668798
[17]	valid_0's auc: 0.668955
[18]	valid_0's auc: 0.669146
[19]	valid_0's auc: 0.66935
[20]	valid_0's auc: 0.66954
[21]	valid_0's auc: 0.669704
[22]	valid_0's auc: 0.66986
[23]	valid_0's auc: 0.670021
[24]	valid_0's auc: 0.670139
[25]	valid_0's auc: 0.670286
[26]	valid_0's auc: 0.67038
[27]	valid_0's auc: 0.670495
[28]	valid_0's auc: 0.670596
[29]	valid_0's auc: 0.670722
[30]	valid_0's auc: 0.670812
[31]	valid_0's auc: 0.670916
[32]	valid_0's auc: 0.670997
[3

## 生成答案

In [23]:
ents_model = load('models/lgb_v5.joblib')
# feature_names = ["query_len", "title_len", "len_ratio", "nt", "per_ratio", "sumper","maxper", "minper", 
#                   "per_ratio_q", "sumper_q","maxper_q", "minper_q", 
# "cos", "euc", "edit", "edit_ratio1", "edit_ratio2", "edit_ratio3", "pearson", "spearman","kendall",
# "inter_num", "inter_ratio1", "inter_ratio2", "inter_tfidf5", "inter_tfidf3"]
# print(len(ents_model.feature_importance()),"个特征")
# print(pd.DataFrame({'column': feature_names,
# 'importance': ents_model.feature_importance(),}).sort_values(by='importance',ascending=False))

In [14]:
test = pd.read_csv('/home/kesci/input/bytedance/first-round/test.csv',header=-1)
np_test_file = np.array(test)#np.ndarray()
test_list=np_test_file.tolist()#list

In [24]:
score = ents_model.predict(x_test, num_iteration=ents_model.best_iteration)

In [25]:
res = []
print(len(score),len(test_list))
for split_line,s in zip(test_list,score):
    query_id = split_line[0]
    title_id = split_line[2]
    res.append([query_id,title_id,s])

5000000 5000000


In [26]:
res_csv = pd.DataFrame(res)

In [27]:
res_csv

,0,1,2
0,1,3,0.393948
1,1,1,0.400944
2,1,4,0.412870
3,1,2,0.376557
4,2,1,0.443438
5,2,2,0.465466
6,2,3,0.450194
7,3,6,0.191502
8,3,5,0.263310
9,3,8,0.228779


In [28]:
res_csv.to_csv('res/submission10.csv',index=False,header=False)

In [ ]:
!./kesci_submit -token fc1ed93da08d69cc -file res/submission10.csv

Kesci Submit Tool
Result File: res/submission10.csv (135.83 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d1072df0a628a002bf7d154","Competition":"5cc51043f71088002c5b8840","Team":"5cfe6ca79b4798002ba303aa","UploadDate":"2019-06-24T06:51:11.484Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.82.121.235","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"1561359057318f96184.csv","ResultFileRealName":"submission10.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}

